In [ ]:
import sys
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K

from tensorflow.keras import layers

## 1. Load datasets

In [ ]:
data_training = '../input/coral-reef-texturestructure/DATA SET/training/Structure-rsmas'
data_validation = '../input/coral-reef-texturestructure/DATA SET/validation/Structure-rsmas'

In [ ]:
from skimage import filters

training_datagen = ImageDataGenerator(
    rescale=1,
    #rescale=1/255,
    shear_range=0.2,
    brightness_range=[0.4,1.0],
    rotation_range=30,
    fill_mode="reflect",
    zoom_range=0.5,
    horizontal_flip=True,
)               

training_generator = training_datagen.flow_from_directory(
    data_training,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
)

In [ ]:
from PIL import Image
import numpy as np
from skimage.transform import resize

def getdata(root):
    folders = glob.glob(os.path.join(root, "*"))
    folders = sorted(folders, key=os.path.basename)
    files, labels = [], []
    
    for _class_idx, folder in enumerate(folders):
        _files = glob.glob(os.path.join(folder, "*"))
        _files = sorted(_files, key=os.path.basename)
        files.extend(_files)
        labels.extend(_class_idx * np.ones(len(_files)))
    
    return files, labels

filenames = getdata(data_validation)

validation_images = []
for file in filenames[0]:
    im = Image.open(file)
    #im = np.array(im)/255.
    im = np.array(im)
    
    im = resize(im, (224, 224, 3), anti_aliasing=True) * 255.
    
    validation_images.append(im)

validation_images = np.stack(validation_images, axis=0)
validation_labels = np.array(filenames[1])

## 2. Display dataset

In [ ]:
import matplotlib.pyplot as plt

image, label = next(training_generator)

nOfimages = 8
for i in range(nOfimages):
    plt.imshow(image[i,...]/255)
    print(np.max(image[i,...]/255))
    plt.colorbar()
    plt.show()
    
    print("class " + str(label[i]))

## 3. Defining the neural network

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.applications.resnet50 import preprocess_input

inputs = tf.keras.Input(shape=(224,224,3))


pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)
pretrained_model.trainable = False

layer = preprocess_input(inputs)
layer = pretrained_model(layer)
layer = tf.keras.layers.Dropout(0.2)(layer)
layer = tf.kera s.layers.GlobalAveragePooling2D()(layer)
layer = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(layer)
layer = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(layer)
output = tf.keras.layers.Dense(14, activation='softmax')(layer)

model = tf.keras.models.Model(inputs, output)
model.summary()

In [ ]:
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## 4. Training

In [ ]:
import tensorflow.keras as keras

_WITH_WEIGHTS_ = True
class_weight = {0: 5,
                1: 6,
                2: 7,
                3: 8,
                4: 9,
                5: 9,
                6: 7,
                7: 7,
                8: 7,
                9: 8,
                10:7,
                11:8,
                12:7,
                13:8}

from pathlib import Path

checkpoint_filepath = '/kaggle/working/checkpoints'
Path(checkpoint_filepath).mkdir(parents=True, exist_ok=True)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

h=model.fit(
    training_generator,
    epochs=60,
    validation_data=(validation_images, validation_labels),
    class_weight=class_weight if _WITH_WEIGHTS_ else None,
    callbacks=[model_checkpoint_callback],
)



## 5. Loss fuction plot

In [ ]:
plt.plot(h.history["loss"], "b", label="Train")
plt.plot(h.history["val_loss"], "g", label="Validation")
plt.grid("on", alpha=0.2)
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.title("Xception + DNN without weight model")
plt.savefig("Loss-xception-true.svg")

## 6. Accuracy plot

In [ ]:
plt.plot(h.history["accuracy"], "b", label="Train")
plt.plot(h.history["val_accuracy"], "g", label="Validation")
plt.grid("on", alpha=0.2)
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.title("InceptionResNetV2 + DNN with weight model")
plt.savefig("Accuracy-xception-true.svg")

## 8. Fine-tuning

In [ ]:
model.layers[3].trainable = True     

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001/10),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

h1= model.fit(
    training_generator,
    epochs=480,
    validation_data=(validation_images, validation_labels),
    class_weight=class_weight,
    callbacks=[model_checkpoint_callback],
)

## 9. Save model

In [ ]:
# Save the entire model
name = "ResNet50"
model.save(name)

In [ ]:
import shutil
shutil.make_archive(name + "_model", 'zip', name)

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(h1.history["loss"], "b", label="Train")
plt.plot(h1.history["val_loss"], "g", label="Validation")
plt.grid("on", alpha=0.2)
plt.ylabel("Loss", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.legend()
plt.legend(prop={'size': 14})
plt.title("ResNet50 + DNN with weight model", fontsize=14)
plt.savefig("Loss-ResNet50-true-after.svg")

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(h1.history["accuracy"], "b", label="Train")
plt.plot(h1.history["val_accuracy"], "g", label="Validation")
plt.ylim([0.6, 1.05])
plt.grid("on", alpha=0.2)
plt.ylabel("Accuracy", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.legend(prop={'size': 14})
plt.title("ResNet50 + DNN with weight model", fontsize=14)
plt.savefig("Accuracy-ResNet50-true-after.svg")

In [ ]:
import pandas as pd

history = {
    "loss": h1.history["loss"], 
    "val_loss": h1.history["val_loss"], 
    "accuracy": h1.history["accuracy"],
    "val_accuracy": h1.history["val_accuracy"]
}
hdf = pd.DataFrame(data=history)
hdf

compression_opts = dict(method='zip',
                        archive_name=name+'_thistory.csv')
hdf.to_csv(name+'_thistory.zip', index=False,
          compression=compression_opts)

## 10. Evaluation

In [ ]:
model.load_weights(checkpoint_filepath)  

predictions = model.predict(validation_images)    
predictions = np.argmax(predictions, axis=1)

In [ ]:
import pandas as pd

data = {"labels": validation_labels, "predictions": predictions}
df = pd.DataFrame(data=data)
df

In [ ]:
compression_opts = dict(method='zip',
                        archive_name=name+'.csv')
df.to_csv(name+'.zip', index=False,
          compression=compression_opts)

## 11. Confusion Matrix Plot

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(validation_labels, predictions)
print("accuracy: " + str(accuracy*100) + "%")


In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,

                           normalize=False,
                           title='Confusion matrix',
                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
     if normalize:
         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
         print("Normalized confusion matrix")
     else:
         print('Confusion matrix, without normalization')

     print(cm)
     plt.imshow(cm, interpolation='nearest', cmap=cmap)
     plt.title(title)
     plt.colorbar()
     tick_marks = np.arange(len(classes))
     plt.xticks(tick_marks, classes, rotation=45)
     plt.yticks(tick_marks, classes)
     fmt = '.2f' if normalize else 'd'
     thresh = cm.max() / 2.
     for i, j in itertools.product(range(cm.shape[0]),   range(cm.shape[1])):
         plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')